In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math
import scipy as sc

In [45]:
class Station():
    def __init__(self, station_name, num_bikes, num_docks, arr_stream):
        
        #Initialization of Station Class:
        #station_name: Name of station (string)
        #num_bikes: number of bikes at the station initially (int)
        #num_docks: number of docks at the station initially (int)
        #arr_stream: data stream containing our arrivals to the station (Pandas DataFrame)
        
        self.station_name = station_name
        self.num_bikes = num_bikes
        self.num_docks = num_docks
        self.arr_stream = arr_stream
        self.simulate(num_bikes, num_docks, arr_stream)
        print("Station " + station_name + " has been initialized")
        
    def simulate(self, num_bikes, num_docks, arr_stream):
        
        ##Initialization Information:##
        
        num_bikes = num_bikes #initial number of bikes
        num_docks = num_docks #initial number of docks
        arr_stream = arr_stream #import arrival stream
        arr_stream["Cust_ID"] = np.arange(len(arr_stream)) #Create customer ID for each customer
        
        ##Initialize queues and arrays to store information##
        
        #Bike_Array: Keeps track of the number of bikes
        num_bike_arr = []        
        #Dock_Array: Keeps track of the number of docks
        num_dock_arr = []
        
        #Borrowing_Queue
        borrowing_queue_length_arr = [] #Array that keeps track of the length of the borrowing queue        
        borrowing_queue_length = 0 #Length of the borrowing queue
        borrowing_queue_customer_ID = [] #keeps track of borrowing customers who join the borrowing queue
        borrowing_queue_customer_in_time = [] #keeps track of the time borrowing customers join the borrowing queue
        borrowing_queue_customer_out_time = [] #keeps track of the time borrowing customers exit the borrowing queue
        
        #Returning_Queue
        returning_queue_length_arr = [] #Array that keeps track of the length of the borrowing queue        
        returning_queue_length = 0 #Length of the borrowing queue
        returning_queue_customer_ID = [] #keeps track of borrowing customers who join the borrowing queue
        returning_queue_customer_in_time = [] #keeps track of the time borrowing customers join the borrowing queue
        returning_queue_customer_out_time = [] #keeps track of the time borrowing customers exit the borrowing queue
        
        #Instant_Service
        instant_service_customer_ID = [] #Keeps track of the customers who get instant service
    
        for i in range(len(arr_stream)): #For all the values in our data stream
            if i == 0: #For first customer
                if arr_stream.iloc[0, :]["Type_of_Customer"] == "Borrowing": #If the customer is borrowing bikes
                    
                    if num_bikes == 0: #If the no bikes to borrow
                        #Number of bikes and docks remain the same
                        borrowing_queue_customer_ID.append(arr_stream.iloc[0, :]["Cust_ID"]) #Add ID to borrowing queue
                        borrowing_queue_customer_in_time.append(arr_stream.iloc[0, :]["Arrival_Time"]) #Add Arrival_Time
                        borrowing_queue_length += 1 #borrowing queue length increases by 1
                        borrowing_queue_length_arr.append(borrowing_queue_length) #append borrowing queue length to array
                    else: #There are bikes to borrow
                        instant_service_customer_ID.append(arr_stream.iloc[0, :]["Cust_ID"]) #Append Cust_ID to instant service array
                        num_bikes = num_bikes - 1 #Customer borrows bike (num_bike goes down by 1)
                        num_docks = num_docks + 1 #Customer borrows bike (num_dock goes up by 1)
                        num_bike_arr.append(num_bikes) #Append number of bikes to array
                        num_dock_arr.append(num_docks) #Append number of docks to array
                        
                else: #If the customer is returning bikes
                    if num_docks == 0: #If there are no docks to return
                        returning_queue_customer_ID.append(arr_stream.iloc[0, :]["Cust_ID"]) #Add ID to returning queue
                        returning_queue_customer_in_time.append(arr_stream.iloc[0, :]["Arrival_Time"]) #Add Arrival_Time
                        returning_queue_length = 1 #returning queue length increases by 1
                        returning_queue_length_arr.append(returning_queue_length) #append returning queue length to array
                    else: #There are docks available
                        instant_service_customer_ID.append(arr_stream.iloc[0, :]["Cust_ID"]) #Append Cust_ID to instant service array
                        num_bikes = num_bikes + 1 #Customer returns bike (num_bike goes up by 1)
                        num_docks = num_docks - 1 #Customer returns bike (num_dock goes down by 1)
                        num_bike_arr.append(num_bikes) #Append number of bikes to array
                        num_dock_arr.append(num_docks) #Append number of docks to array
            else:
                
                if arr_stream.iloc[i, :]["Type_of_Customer"] == "Borrowing": #If customer is borrowing
                    
                    if borrowing_queue_length > 0: #Borrowing queue has people
                        #Try to servce queued customers first
                        if returning_queue_length > 0: #If returning queue has people
                            returning_queue_length -= 1 #Decrease the returning queue individual by one
                            borrowing_queue_length -= 1 #Decrease the borrowing queue individuals by one
                            returning_queue_length_arr.append(returning_queue_length) #Append queue length                            
                            returning_queue_out_time.append(arr_stream.iloc[i, :]["Arrival_Time"]) #Update exit time for returning_queue customer
                            borrowing_queue_out_time.append(arr_stream.iloc[i, :]["Arrival_Time"]) #Update exit time for borrowing_queue customer
                            
                            #Decrease number of bike by one and increase dock by one
                            num_bikes = num_bikes - 1 #Customer borrows bike (num_bike goes down by 1)
                            num_docks = num_docks + 1 #Customer borrows bike (num_dock goes up by 1)
                            num_bike_arr.append(num_bikes) #Append number of bikes to array
                            num_dock_arr.append(num_docks) #Append number of docks to array
                            
                            #Add current customer to borrowing_queue
                            borrowing_queue_customer_ID.append(arr_stream.iloc[i, :]["Cust_ID"]) #Add ID to borrowing queue
                            borrowing_queue_customer_in_time.append(arr_stream.iloc[i, :]["Arrival_Time"]) #Add Arrival_Time
                            borrowing_queue_length += 1 #borrowing queue length increases by 1
                            borrowing_queue_length_arr.append(borrowing_queue_length) #append length to borrowing_queue
                            
                        else: #borrowing queue has no people
                            
                            borrowing_queue_customer_ID.append(arr_stream.iloc[i, :]["Cust_ID"])#Add customer ID to borrowing queue
                            borrowing_queue_customer_in_time.append(arr_stream.iloc[i, :]["Arrival_Time"]) #Add Arrival_Time
                            borrowing_queue_length += 1 #returning queue length increases by 1
                            borrowing_queue_length_arr.append(borrowing_queue_length) #append returning queue length to array
                            
                    elif borrowing_queue_length == 0 and num_bikes > 0: #If there is no borrowing queue and there are bikes available
                        
                        #Instant_Service
                        instant_service_customer_ID.append(arr_stream.iloc[i,:]["Cust_ID"]) #Append Cust_ID to instant service array
                        
                        ##TEST##
                        #returning_queue_customer_out_time.append(arr_stream.iloc[i,:]["Arrival_Time"])
                        
                        num_bikes = num_bikes - 1 #Customer borrows bike (num_bike goes down by 1)
                        num_docks = num_docks + 1 #Customer borrows bike (num_dock goes up by 1)
                        num_bike_arr.append(num_bikes) #Append number of bikes to array
                        num_dock_arr.append(num_docks) #Append number of docks to array

                    elif num_bikes == 0: #No bikes to borrow
                        
                        #Add to borrowing queue
                        borrowing_queue_customer_ID.append(arr_stream.iloc[i, :]["Cust_ID"])#Add customer ID to borrowing queue
                        borrowing_queue_customer_in_time.append(arr_stream.iloc[i, :]["Arrival_Time"]) #Add Arrival_Time
                    
                        
                else: #If customer is returning
                    if returning_queue_length > 0: #Returning queue has people
                    #Try to servce queued customers first
                        
                        if returning_queue_length > 0: #If returning queue has people
                            returning_queue_length -= 1 #Decrease the returning queue individual by one
                            borrowing_queue_length -= 1 #Decrease the borrowing queue individual by one
                            borrowing_queue_length_arr.append(borrowing_queue_length) #Append queue length                            
                            returning_queue_customer_out_time.append(arr_stream.iloc[i, :]["Arrival_Time"]) #Update exit time for returning_queue customer
                            borrowing_queue_customer_out_time.append(arr_stream.iloc[i, :]["Arrival_Time"]) #Update exit time for borrowing_queue customer
                            #Increase number of bike by one and decrease dock by one
                            num_bikes = num_bikes + 1 #Customer returns bike (num_bike goes up by 1)
                            num_docks = num_docks - 1 #Customer borrows bike (num_dock goes down by 1)
                            num_bike_arr.append(num_bikes) #Append number of bikes to array
                            num_dock_arr.append(num_docks) #Append number of docks to array

                            #Add current customer to returning_queue
                            returning_queue_customer_ID.append(arr_stream.iloc[i, :]["Cust_ID"]) #Add ID to borrowing queue
                            returning_queue_customer_in_time.append(arr_stream.iloc[i, :]["Arrival_Time"]) #Add Arrival_Time
                            returning_queue_length += 1 #returning queue length increases by 1
                            returning_queue_length_arr.append(returning_queue_length) #append length to returning_queue

                        elif borrowing: #returning queue has no people
                            returning_queue_customer_ID.append(arr_stream.iloc[i, :]["Cust_ID"])#Add customer ID to returning queue
                            returning_queue_customer_in_time.append(arr_stream.iloc[i, :]["Arrival_Time"]) #Add Arrival_Time
                            returning_queue_length += 1 #returning queue length increases by 1
                            returning_queue_length_arr.append(returning_queue_length) #append returning queue length to array

                    elif returning_queue_length == 0 and num_docks > 0: #If there is no returning queue and there are bikes available
                        #Instant_Service
                        instant_service_customer_ID.append(arr_stream.iloc[i, :]["Cust_ID"]) #Append Cust_ID to instant service array
                        
                        ##TEST##
                        #instant_queue_customer_out_time.append(arr_stream.iloc[i,:]["Arrival_Time"])
                        
                        num_bikes = num_bikes + 1 #Customer returns bike (num_bike goes up by 1)
                        num_docks = num_docks - 1 #Customer borrows bike (num_dock goes down by 1)
                        num_bike_arr.append(num_bikes) #Append number of bikes to array
                        num_dock_arr.append(num_docks) #Append number of docks to array

                    elif num_docks == 0: #No docks to return bikes to
                        #Add to returning queue
                        returning_queue_customer_ID.append(arr_stream.iloc[i, :]["cust_ID"])#Add customer ID to borrowing queue
                        returning_queue_customer_in_time.append(arr_stream.iloc[i, :]["Arrival_Time"]) #Add Arrival_Time
             
        self.num_bike_arr = num_bike_arr
        self.num_dock_arr = num_dock_arr
        self.borrowing_queue_length_arr = borrowing_queue_length_arr
        self.returning_queue_length_arr = returning_queue_length_arr
        
        self.borrowing_queue_customer_ID = borrowing_queue_customer_ID
        self.borrowing_queue_customer_in_time = borrowing_queue_customer_in_time
        self.borrowing_queue_customer_out_time = borrowing_queue_customer_out_time
        
        self.returning_queue_customer_ID = returning_queue_customer_ID
        self.returning_queue_customer_in_time = returning_queue_customer_in_time
        self.returning_queue_customer_out_time = returning_queue_customer_out_time
        
        self.instant_service_customer_ID = instant_service_customer_ID
        
    def get_num_bike_arr(self):
        return self.num_bike_arr
    
    def get_num_dock_arr(self):
        return self.num_dock_arr
    
    def get_borrowing_queue_lengths(self):
        return self.borrowing_queue_length_arr
    
    def get_returning_queue_lengths(self):
        return self.returning_queue_length_arr
    
    def get_borrowing_queue(self):
        #If in time is less than out time, add 
        len_diff = len(self.borrowing_queue_customer_in_time) - len(self.borrowing_queue_customer_out_time)
        print(self.borrowing_queue_customer_in_time)
        print(self.borrowing_queue_customer_out_time)
        for i in np.arange(len_diff):
            self.borrowing_queue_customer_out_time.append("Not Served")   
        borrowing_df = pd.DataFrame({"Cust_ID": self.borrowing_queue_customer_ID,
                     "Cust_In_Time": self.borrowing_queue_customer_in_time,
                     "Cust_Out_Time": self.borrowing_queue_customer_out_time})        
        return borrowing_df
    
    def get_returning_queue(self):
        #If in time is less than out time, add 
        len_diff = len(self.returning_queue_customer_in_time) - len(self.returning_queue_customer_out_time)
        for i in np.arange(len_diff):
            self.returning_queue_customer_out_time.append("Not Served")
            
        returning_df = pd.DataFrame({"Cust_ID": self.returning_queue_customer_ID,
                                    "Cust_In_Time": self.returning_queue_customer_in_time,
                                    "Cust_Out_Time": self.returning_queue_customer_out_time})        
        return returning_df
    
    def get_instant_service_customer_queue(self):
        return self.instant_service_customer_ID #Keeps track of the customers who get instant service


In [46]:
test_df_0 =  pd.DataFrame({"Arrival_Time": [0, 1],
                        "Type_of_Customer": ["Borrowing", "Returning"],
                        "Station_From": ["Joe", "Joe"]
                       })
test_df_0


,Arrival_Time,Type_of_Customer,Station_From
0,0,Borrowing,Joe
1,1,Returning,Joe


In [47]:
test_df = pd.DataFrame({"Arrival_Time": [0, 1, 2, 3, 4, 5],
                        "Type_of_Customer": ["Returning", "Borrowing", "Returning", "Borrowing", "Returning", "Borrowing"],
                        "Station_From": ["Joe", "Joe", "Joe", "Joe", "Joe", "Joe"]
                       })
test_df

,Arrival_Time,Type_of_Customer,Station_From
0,0,Returning,Joe
1,1,Borrowing,Joe
2,2,Returning,Joe
3,3,Borrowing,Joe
4,4,Returning,Joe
5,5,Borrowing,Joe


In [48]:
test_station = Station("Joe", 5, 0, test_df_0)

Station Joe has been initialized


In [49]:
test_station.get_returning_queue()

,Cust_ID,Cust_In_Time,Cust_Out_Time


In [50]:
test_station.get_borrowing_queue()

[]
[]


,Cust_ID,Cust_In_Time,Cust_Out_Time


In [51]:
test_station.get_returning_queue_lengths()

[]

In [52]:
test_station.get_instant_service_customer_queue()

[0, 1]